In [1]:
!pip install transformers

In [2]:
!pip install -U sagemaker

  Using cached sagemaker-2.248.2-py3-none-any.whl.metadata (17 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
Using cached sagemaker-2.248.2-py3-none-any.whl (1.7 MB)
Using cached attrs-25.3.0-py3-none-any.whl (63 kB)
  Attempting uninstall: attrs
    Found existing installation: attrs 23.2.0
    Uninstalling attrs-23.2.0:
      Successfully uninstalled attrs-23.2.0
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.245.0
    Uninstalling sagemaker-2.245.0:
      Successfully uninstalled sagemaker-2.245.0━━━━━━━━━━━━━━━━━━ 1/2 [sagemaker]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [sagemaker]/2 [sagemaker]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker-studio-analytics-extension 0.2.0 requires sparkmagic==0.22.0, but you have sparkmagic 0.21.0 which is incompatible.
sparkmagic 0.21.0 requires pandas

In [3]:
!pip install s3fs

In [10]:
!pip install loguru

In [ ]:
import sagemaker
from sagemaker.huggingface import HuggingFace
import boto3
import os
from dotenv import load_dotenv

# --- Configuration ---
load_dotenv()

In [ ]:
# IAM role for SageMaker
iam_role = os.getenv("SAGEMAKER_IAM_ROLE")
# S3 bucket for data and model artifacts
s3_bucket = os.getenv("S3_BUCKET")  

# try:
# 	role = sagemaker.get_execution_role()
#     # role = "arn:aws:iam::551529993308:role/service-role/AmazonSageMaker-ExecutionRole-20250711T075198"
# except ValueError:
# 	iam = boto3.client('iam')
# 	role = iam.get_role(RoleName='sagemaker_execution_role')['Role']['Arn']


In [ ]:
# Define S3 paths
base_s3_uri = f"s3://{s3_bucket}"
base_model_s3_uri = f"{base_s3_uri}/self-corrective-llm-not-trained"
dataset_s3_uri = f"{base_s3_uri}/datasets/training_data"
output_s3_uri = f"{base_s3_uri}/trained_model/output"

In [ ]:
# --- Hyperparameters ---
hyperparameters = {
    "epochs": 1,
    "train_batch_size": 2,
    "eval_batch_size": 2,
    "learning_rate": 2e-4,
    "alpha": 0.7,
}

# --- SageMaker Estimator ---
huggingface_estimator = HuggingFace(
    entry_point="train.py",          # Your training script
    source_dir="../../scripts",         # Directory containing the script
    instance_type="ml.g5.2xlarge",   # Instance type for training
    instance_count=1,
    role=iam_role,
    transformers_version="4.36",     # Version of transformers
    pytorch_version="2.1",           # Version of PyTorch
    py_version="py310",              # Python version
    hyperparameters=hyperparameters,
    output_path=output_s3_uri,
    # Dependencies from your project
    dependencies=["../src"],
    # Input channels for data and base model
    sagemaker_session=sagemaker.Session(),
    # input_mode='File',
#     distribution={"torch_distributed": {"enabled": True}}
)

In [ ]:
# --- Start Training ---
huggingface_estimator.fit({
    "dataset": dataset_s3_uri,
    "model": base_model_s3_uri
})

print("SageMaker training job started.")
